In [240]:
import streamlit as st
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import pandas as pd

df_movies=pd.read_csv(r"C:\Users\Admin\ml_data.csv")


In [242]:
df_movies

,tconst,originalTitle,startYear,runtimeMinutes,averageRating,numVotes,actors_actresses,directors,writers,ranking,genres,overview,poster_path
0,tt0111161,The Shawshank Redemption,1994,142.0,9.3,2832004.0,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Frank Darabont,Stephen King,341.419393,"Drama,Crime",Framed in the 1940s for the double murder of h...,/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg
1,tt0468569,The Dark Knight,2008,152.0,9.0,2813352.0,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",Christopher Nolan,"Jonathan Nolan, David S. Goyer, Bob Kane",325.373123,"Drama,Action,Crime,Thriller",Batman raises the stakes in his war on crime. ...,/qJ2tW6WMUDux911r6m7haRef0WH.jpg
2,tt1375666,Inception,2010,148.0,8.8,2497591.0,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",Christopher Nolan,NaN,279.912006,"Action,ScienceFiction,Adventure","Cobb, a skilled thief who commits corporate es...",/edv5CZvWj09upOsy2Y6IwDhK8bt.jpg
3,tt0137523,Fight Club,1999,139.0,8.8,2267684.0,"Brad Pitt, Edward Norton, Meat Loaf, Zach Grenier",David Fincher,"Chuck Palahniuk, Jim Uhls",252.853483,"Drama,Thriller,Comedy",A ticking-time-bomb insomniac and a slippery s...,/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg
4,tt0110912,Pulp Fiction,1994,154.0,8.9,2171750.0,"John Travolta, Samuel L. Jackson, Bruce Willis...",Quentin Tarantino,Roger Avary,250.725992,"Thriller,Crime","A burger-loving hit man, his philosophical par...",/d5iIlFn5s0ImszYzBPb8JPIfbXD.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49009,tt0133157,PNYC: Portishead - Roseland New York,1997,93.0,8.9,621.0,NaN,Dick Carruthers,NaN,-14.020452,"Music,Documentary","Portishead concert in the Roseland Ballroom, N...",/kRX3LLAtOuRT8qZ4qKUgUvH6Hrh.jpg
49010,tt0232152,Nagara Haavu,1972,184.0,9.0,1160.0,"Ambarish, K.S. Ashwath, Arathi, M.N. Lakshmi Devi",S.R. Puttana Kanagal,"Ta. Ra. Subba Rao, Chi Udayashankar",-14.123986,Drama,Although Ramachari falls madly in love with Al...,/iQWIDF4vRDG7LBl4SOPpgpuTIIK.jpg
49011,tt1935109,Dipu Number 2,1996,154.0,8.9,2540.0,"Shafquat, Arun Saha, Shubashish, Bulbul Ahmed,...",Morshedul Islam,Muhammad Zafar Iqbal,-14.435537,"Adventure,Family","An adventure story for young boys, Dipu Number...",/4Gz1HkpZhwWpyuual7eytb5CBj9.jpg
49012,tt0383177,Aguner Poroshmoni,1994,123.0,9.1,3382.0,"Asaduzzaman Noor, Abul Hayat, Bipasha Hayat, D...",Humayun Ahmed,NaN,-14.668931,"Drama,War",A middle-class family shelters a freedom fight...,/ygJopYcwwi8wbZTaUFav5u6Rz8L.jpg


In [237]:
st.title("Movie Recommendation System")
selected_movie = st.selectbox("Select a movie for recommendations:", df_movies['originalTitle'].unique())
num_recommendations = st.slider("Select the number of recommendations:", 1, 10, 6)

def get_recommendations(movie_title,num_recs):
    top30000 = df_movies.head(5000).reset_index(drop=True)

    index_movie = top30000.loc[top30000["originalTitle"] == selected_movie].index[0]
    def splitter(value):
        return value.split(',') if isinstance(value, str) else []

    actors_in_chosen_film = splitter(top30000.iloc[index_movie]['actors_actresses'])
    directors_in_chosen_film = splitter(top30000.iloc[index_movie]['directors'])
    writers_in_chosen_film = splitter(top30000.iloc[index_movie]['writers'])
    genres = splitter(top30000.iloc[index_movie]['genres'])

    def create_dummy_columns(df, names_list, column_name, df_column):
        for name in names_list:
            if name:
                name = name.strip()
                df[column_name + name] = df[df_column].apply(
                    lambda x: 1 if isinstance(x, str) and name in x else 0)

    create_dummy_columns(top30000, actors_in_chosen_film, 'actor_', 'actors_actresses')
    create_dummy_columns(top30000, directors_in_chosen_film, 'director_', 'directors')
    create_dummy_columns(top30000, writers_in_chosen_film, 'writer_', 'writers')
    create_dummy_columns(top30000, genres, 'genre_', 'genres')

             
    df_knn = top30000.drop(columns=['tconst', 'genres','overview','poster_path', 'originalTitle', 'actors_actresses', 'directors', 'writers','averageRating','numVotes'])
    X = df_knn

    weight=1
    for col in df_knn.columns:
        if col.startswith('genre_'):
            df_knn[col] *= weight

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = NearestNeighbors(n_neighbors=10)
    model.fit(X_scaled)

    recommendation = model.kneighbors([X_scaled[index_movie], ])[1][0]
    recommended_movies = top30000.iloc[recommendation][['originalTitle', 'runtimeMinutes', 'genres', 'startYear', 'averageRating', 'numVotes', 'actors_actresses', 'directors', 'overview', 'poster_path']]
    recommended_movies = recommended_movies.set_index('originalTitle')

    return recommended_movies.head(num_recs)

if st.button("Get Recommendations"):
    recommendations = get_recommendations(selected_movie, num_recommendations)

    col_index = 0
    cols = [None, None, None] 

    for index, (title, row) in enumerate(recommendations.iterrows()):
        if index % 3 == 0:
            cols = st.columns(3) 

        with cols[index % 3]:
            st.subheader(title)
            st.image('https://image.tmdb.org/t/p/original/'+row['poster_path'], width=200)
            st.write(row['overview']) 



